# Защита персональных данных клиентов

Нам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обосновать корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
df = pd.read_csv('/datasets/insurance.csv')

In [3]:
display(df.head(5))
df.info()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [4]:
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [5]:
features = df.drop(['Страховые выплаты'], axis=1)  # извлекаем признаки 
target = df['Страховые выплаты'] # извлекаем целевой признак

In [6]:
X = np.concatenate((np.ones((features.shape[0], 1)), features), axis=1)
y = target
w = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
w = w[1:]
w0 = w[0]
print(w)
print(w0)

model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
print('Значение r2:', r2_score(target, predictions))
model.coef_

[ 7.92580543e-03  3.57083050e-02 -1.70080492e-07 -1.35676623e-02]
0.007925805428953745
Значение r2: 0.4249455028666801


array([ 7.92580543e-03,  3.57083050e-02, -1.70080492e-07, -1.35676623e-02])

Вывод: 
В датасете указаны данные клиентов которые былли застрахованы, за целевой признак берем сколько стрраховых выплат получил клиент.

Аномалии в данных не выявлено.

Обучили линейную регрессию. Вычитали значения и R2.

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Не измениться

**Обоснование:** Вставим умножение на обратимую матрицу P, чтобы доказать, что : $a'=a$

$$ a'= XP((XP)^T XP)^{-1} (XP)^T y $$

Для следующего шага понадобиться следующее свойство обатимой матрицы: $$ (AB)^T = A^T B^T $$

$$
a' = XP (P^T X^T XP)^{-1} P^T X^T y
$$

Для следующего шага понадобиться следующее свойство транспонированной матрицы:
$$
(AB)^{-1} = B^{-1} A^{-1} 
$$

$$ a'= XP(P)^{-1} (X^TX)^{-1} (P^T)^{-1} X^T P^T y $$
Раскроем скобки: $$ a'= XP(P)^{-1} (X^TX)^{-1} (P^T)^{-1} X^T P^T y $$

Умножение $$PP^{-1} = E$$

Умножением на единичную матрицу не чего не меняет: 
$$ a' = XE (X^TX)^{-1} EX^T y $$

$$ a' = X (X^TX)^{-1} X^T y $$

$$ a' = Xw $$

$$a'=a$$

Как видно, значение предсказания  не меняется, если умножать матрицу признаков на обратимую матрицу.

## Алгоритм преобразования

**Алгоритм**

Для защиты информации будем умножать матрицу признаков на обратимую матрицу , которая будет генерироваться случайным образом.

Этапы алгоритма:

1.Сгенерировать случайную матрицу размером 4 на 4.

2.Проверить сгенерированную матрицу на обратимость.

3.Получение матрицы преобразованных признаков, путем умножения исхожной матрицы на сгенерированную.

4.Применение алгоритма на преобразованных признаках.

5.Сравним метрики R2 на преобразованных и на исходных данных.

**Обоснование**

Матрица B должна иметь необходимую размерность xnx, где n - количество признаков для регрессии. Таким образом матрица C будет иметь туже размерность, что и матрица A. Обратная матрица B существует только для квадратных невырожденных матриц (определитель которых не равен нулю).

## Проверка алгоритма

In [7]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=1234567)

In [8]:
model = LinearRegression()
model.fit(features_train, target_train)
predictions = model.predict(features_test)
r2 = r2_score(target_test, predictions)
print(model.coef_)
print("Значение r2:", r2)

[ 1.68387835e-02  3.62788008e-02  7.59473675e-08 -1.25231671e-02]
Значение r2: 0.4129570107995183


In [9]:
def new_features(features):
      
    # делаем генератор случайных чисел:
    np.random.seed(1234567)
    # создаем массив случайных чисел из интервала:
    new_matrix = np.random.randint(1, 10, (features.shape[1],features.shape[1]))
    # вычисляем детерминант матрицы
    det = np.linalg.det(new_matrix)
    while det == 0:
        np.random.seed(1234568)
        new_matrix = np.random.randint(1, 10, (features.shape[1],features.shape[1]))
    
    new_features = features @ new_matrix
    return new_features, new_matrix

Вывел матрицу на экран и вижу, что данные потеряли свои оригинальные значения, теперь в них хранится непонятные числа. 

In [10]:
features, new_matrix = new_features(features)
display(features.head())
new_matrix

,0,1,2,3
0,49730.0,148854.0,297653.0,198578.0
1,38141.0,114054.0,228051.0,152192.0
2,21087.0,63029.0,126029.0,84116.0
3,41769.0,125137.0,250231.0,166900.0
4,26188.0,78333.0,156635.0,104518.0


array([[4, 5, 7, 6],
       [3, 1, 1, 4],
       [1, 3, 6, 4],
       [3, 8, 5, 8]])

In [11]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=1234567)

In [12]:
# Код ревьюера
display(features_train.head(3))
display(features_test.head(3))

,0,1,2,3
878,17906.0,53453.0,106849.0,71350.0
4996,52505.0,157242.0,314439.0,209744.0
1052,40187.0,120329.0,240629.0,160516.0


,0,1,2,3
3221,47603.0,142545.0,285044.0,190142.0
4078,55426.0,165956.0,331850.0,221376.0
4192,55766.0,167122.0,334222.0,222888.0


In [13]:
model = LinearRegression()
model.fit(features_train, target_train)
predictions_new_table = model.predict(features_test)
r2_new_table = r2_score(target_test, predictions_new_table)
print(model.coef_)
print("Значение r2:", r2_new_table)

[ 0.00890267 -0.00985182  0.00024605  0.00479414]
Значение r2: 0.41295701079950675


## Вывод

Вывод: 
1. Проанализиовали исходные данные, аномалии не было выявленно.

2. Создали матрицу для шифрования исходных данных, преобразовали исходные признаки.

3. Проверили метрику R2 до преобразования и после, качество линейной регрессии не изменилось.